# Lab 2: Tuning a Perceptron Classifier using Cross-Validation and Grid Search

## Introduction

在本课程中,我们将学习如何使用 scikit-learn 的工具来训练Perceptron。我们将重点关注:

1. **了解如何拆分训练集和留出样本(holdout set)，来估计训练误差和泛化误差**
2. **了解ML的参数包括正则化参数和模型超参数**
2. **理解scikit-learn中Pipeline的概念，并学会使用Pipeline来自动化流程**
1. **使用交叉验证(cross-validation)来优化参数**

完成本课程后,你应该能够:

- 在大模型的帮助下完成Pipeline的设定
- 利用交叉验证来优化给定模型的表现

让我们开始吧！

# 理论

## 背景设定

你准备考TOFEL，下面是你可以选择的一些备考方案（Policy）
- 1. 去庙里求个签
- 2. 买本书回家自己刷
- 3. 报个新东方培训班
- 4. 请个私教

### 问题

1. 在这个背景里，你的优化目标是什么？
2. 你的训练损失是什么？你的泛化损失是什么？
3. 在这个背景下，你的feature set是什么？
4. 你通过背答案，在模拟考上得了很高分，但是在真实考试中却一塌胡涂。这种现象在机器学习里叫什么？
5. 为什么有人会花钱去韩国考机考？而不是在国内做笔试？
6. 【挑战题】什么是VC维？(VC维刻画的是什么？)。为什么从1到4的VC维不断升高？


## 1. 加载和探索数据

In [1]:
import pandas as pd
import numpy as np

# 加载数据
data = pd.read_csv('../data/loan_data.csv')

## 2. 数据预处理

在训练模型之前,我们需要预处理数据:

- 编码分类变量
- 标准化数值特征

*JC：如果你看不懂下面的代码无需担心，我们会在lab3中详细讲解*

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 数值和分类特征列表
numerical_features = [
    'person_age', 'person_income', 'person_emp_exp', 'loan_amnt',
    'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
    'credit_score'
]
categorical_features = [
    'person_gender', 'person_education', 'person_home_ownership',
    'loan_intent', 'previous_loan_defaults_on_file'
]

# 定义预处理步骤
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# 分离特征和目标变量
X = data.drop('loan_status', axis=1)
y = data['loan_status']

**说明:**

- **数值特征**: 使用 `StandardScaler` 进行标准化,使其均值为0,方差为1
- **分类特征**: 使用 `OneHotEncoder` 将类别转换为二进制变量

## 3. 将数据分割为训练集和留出集

我们将数据分为训练集和留出(测试)集。留出集将在最后用于评估模型的泛化性能。

*HW：Q1 问大模型为什么要设置random state*

In [7]:
from sklearn.model_selection import train_test_split

# 分割数据
X_train_val, X_holdout, y_train_val, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 4. 使用交叉验证选择参数

### a. 创建管道

*HW：Q2 问大模型使用Pipeline的好处*

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Perceptron

# 定义管道
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', Perceptron(random_state=42))
])

### b. 定义参数
*TODO：问大模型为什么什么叫L1损失，什么情况下应该使用L1损失*

In [9]:
penalty_method = ['l2', 'l1', 'elasticnet']

### c. 执行交叉验证

*TODO：问大模型cv这个参数什么意思，cv参数变大或者变小的影响是什么？*

*HW Q3：如何让你的可视化更加直观？*


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

# 定义正则化参数(alpha)的取值范围

mean_scores = []

for penalty in penalty_method:
    pipeline.set_params(classifier__penalty=penalty)  # pipeline defines the automation process
    scores = cross_val_score(
        pipeline, X_train_val, y_train_val, cv=5, scoring='accuracy')
    mean_scores.append(scores.mean())

# 绘制结果
plt.figure(figsize=(8, 6))
plt.bar(penalty_method, mean_scores)
plt.xlabel('Penalty Method')
plt.ylabel('Cross-validation accuracy')
plt.title('Selecting Penalty Method with cross-validation')
plt.show()

In [ ]:
# 找到具有最高平均分数的alpha
best_penalty = penalty_method[np.argmax(mean_scores)]
print(f'最佳 参数: {best_penalty}, 最佳准确率是{mean_scores[np.argmax(mean_scores)]}')

## 5. 使用网格搜索进行多个超参数选择

### a. 定义参数网格
*TODO：Alpha是什么意思？在这里起了什么作用*

*HW Q4：问大模型eta0是什么意思？*
*HW Challenge1: 不拟合常数项的实际含义是什么？【提示，过哪个定点】*

In [12]:
from sklearn.model_selection import GridSearchCV

# 定义参数网格
param_grid = {
    'classifier__penalty': ['l2', 'l1', 'elasticnet'],
    'classifier__alpha': np.logspace(-5, 1, 7),
    'classifier__eta0': [1.0, 0.1, 0.01],
    'classifier__fit_intercept': [True, False]
}

In [ ]:
grid_search = GridSearchCV(
    pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=3)

grid_search.fit(X_train_val, y_train_val)

print(f'最佳参数: {grid_search.best_params_}')
print(f'最佳交叉验证分数: {grid_search.best_score_:.4f}')

## 7. 评估模型
*ToDo：你发现holdout set的accuracy变差了，这说明什么？首先尝试自己解释，然后让大模型点评你的解释*

*ToDo：反思，alpha的值设定合理么？怎么优化*

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# 获取最佳模型
best_model = grid_search.best_estimator_

# 在留出集上进行预测
y_pred = best_model.predict(X_holdout)

# 评估模型
accuracy = accuracy_score(y_holdout, y_pred)
print(f'留出集上的准确率: {accuracy:.4f}')

print('分类报告:')
print(classification_report(y_holdout, y_pred))

## Now You Do!
**Excercise**

调整alpha来平衡训练误差和泛化误差

**Mini Kaggle**

在random state=42，holdout 比例20%的前提下；综合你在这个lab中学到的技巧，比一比谁的模型能在holdout set中得到表现最好